In [1]:
# general
import warnings, gc, sys, os, operator
warnings.filterwarnings('ignore')

# data
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [ ]:
read_path = '../dataset/'
write_path = '../dataset/fin/'
train = pd.read_csv(read_path+'origin/train.csv')
test = pd.read_csv(read_path+'origin/test.csv')

In [ ]:
train = trainTestFE(train)
test = trainTestFE(test, isTrain=False)
train.to_csv(path+'trainFE_0213.csv', index=False)
test.to_csv(path+'testFE_0213.csv', index=False)

In [ ]:
def trainTestFE(df, isTrain=True):

    df['first_active'] = pd.to_datetime(df['first_active_month'])
    df['first_active_year'] = df['first_active'].dt.year
    df['first_active_month'] = df['first_active'].dt.month
    df['first_active_quarter'] = df['first_active'].dt.quarter
    df['first_active_weekofyear'] = df['first_active'].dt.weekofyear
    df['first_active_dayofweek'] = df['first_active'].dt.dayofweek
    
    # 모든 데이터의 마지막 거래날짜가 2018년 4월 30일 23시 59분 59초
    df['first_active_elapsed_time_from_trade'] = (dt(2018, 4, 30, 23, 59, 59) - df['first_active']).dt.days
    df['first_active_elapsed_time_from_today'] = (dt.today() - df['first_active']).dt.days
    df['first_active_total_day'] = pd.to_timedelta(df['first_active']).dt.total_seconds() / (60 * 60 * 24)

    df['days_feature1_trade'] = df['feature_1'] * df['first_active_elapsed_time_from_trade']
    df['days_feature2_trade'] = df['feature_2'] * df['first_active_elapsed_time_from_trade'] 
    df['days_feature3_trade'] = df['feature_3'] * df['first_active_elapsed_time_from_trade'] 

    df['days_feature1_trade_ratio'] = df['feature_1'] / df['first_active_elapsed_time_from_trade']
    df['days_feature2_trade_ratio'] = df['feature_2'] / df['first_active_elapsed_time_from_trade']
    df['days_feature3_trade_ratio'] = df['feature_3'] / df['first_active_elapsed_time_from_trade']
    
    if isTrain :
        df['outliers'] = (df['target']==df['target'].min())*1
        for f in ['feature_1','feature_2','feature_3', 'first_active_year', 'first_active_month', 'first_active_total_day']:
            order_label = df.groupby([f])['outliers'].mean()
            df[f + '_outlier'] = df[f].map(order_label)

        df['feature_sum_outlier'] = df['feature_1_outlier'] + df['feature_2_outlier'] + df['feature_3_outlier']
        df['feature_mean_outlier'] = df['feature_sum_outlier'] / 3
        df['feature_max_outlier'] = df[['feature_1_outlier', 'feature_2_outlier', 'feature_3_outlier']].max(axis=1)
        df['feature_min_outlier'] = df[['feature_1_outlier', 'feature_2_outlier', 'feature_3_outlier']].min(axis=1)
        df['feature_var_outlier'] = df[['feature_1_outlier', 'feature_2_outlier', 'feature_3_outlier']].std(axis=1)

    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
    df['feature_mean'] = df['feature_sum'] / 3
    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
    df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)
    
    return df